## Modifying parameters in clm-5 and performing a point-scale experiment at TALL-NEON site
### Introduction 
We shall modify the Vcmax parameter in CLM5. You can refer to the CLM5 documentantion for more details and information about the vcmax parameter.

We shall determine the Vcmax parameter value used for the default run and then create two parameter files where we shall modify the default value of the vcmax parameter to its minimum value and maximum values according to literature.
We shall then, perform a simple experiemnt to determine the impacts of the modified parameter on GPP and latent heat flux simulated at TALL sites.

### Getting started

The first step is to copy-paste the original parameter file located here "~/inputdata/lnd/clm2/paramdata/ctsm51_params.c211112.nc". The following code will create two new parameter files which we will edit to modify the given parameter vcmax.

##### Note the %%bash on the beginning of the cell below, this tells jupyter to use bash syntax.

In [ ]:
%%bash
cp /home/user/inputdata/lnd/clm2/paramdata/ctsm51_params.c211112.nc /home/user/inputdata/lnd/clm2/paramdata/ctsm51_params.c211112_vcmax_min.nc
cp /home/user/inputdata/lnd/clm2/paramdata/ctsm51_params.c211112.nc /home/user/inputdata/lnd/clm2/paramdata/ctsm51_params.c211112_vcmax_max.nc

## Loading the parameter files
Here we load all the three parameter files and prepare them for editing. Here we are going to change the parameter vcmaxha. The following cell loads the three parameter file. The default, and the created new files.

In [ ]:
import netCDF4 as nc
#To help us remember what file we are working with, we shall name the imported datasets 
#based on what each file means. Since we are changing the vmax parameter to its minimum
#and maximum values, naming the files this will help us understand what file we are working with
default = '/home/user/inputdata/lnd/clm2/paramdata/ctsm51_params.c211112.nc'
vmax_min = '/home/user/inputdata/lnd/clm2/paramdata/ctsm51_params.c211112_vcmax_min.nc'
vmax_max = '/home/user/inputdata/lnd/clm2/paramdata/ctsm51_params.c211112_vcmax_max.nc'

#Preparing them for editing

default = nc.Dataset(default, 'r+')
vmax_min = nc.Dataset(vmax_min, 'r+')
vmax_max = nc.Dataset(vmax_max, 'r+')

In [ ]:
#To see all the available variables in our created parameter files, we can use the following command.
default.variables

In [ ]:
#Now we need to see the default value for the parameter vcmax. In the model, the vmaxha is the parameter associated
#with the activation energy and it is what we are interested in for this assignment. The following code returns the 
#default parameter value for the vcmaxha
default.variables['vcmaxha'][:]

## Changing the vmaxha value to its maximum and minimum values
According to existing literature, the minimum vmaxha value is 45000 and its maximum value is 173000. The following code will edit the vmax_min and vmax_max files to input the minimum and maximum vmaxha values to each file respectively.

In [ ]:
vmax_min.variables['vcmaxha'][:]=45000
vmax_max.variables['vcmaxha'][:]=173000

#To check if the new values were added, you can use the following code
vmax_min.variables['vcmaxha'][:],vmax_max.variables['vcmaxha'][:]

In [ ]:
#close editing for all the three datasets
vmax_min.close()
vmax_max.close()

## Creating two new cases and adding the modified parameter files and running them for 36 months

In [ ]:
%%bash
cd /opt/ncar/cesm/cime/scripts/
#Creates the first case for the vcmax_min modified minimum parameter
./create_newcase --case /home/user/TALLvcmax_min --compset I1PtClm51Bgc --res CLM_USRDAT --driver nuopc --user-mods-dir NEON/TALL

#Creates the second case for the vcmax_max modified maximum parameter
./create_newcase --case /home/user/TALLvcmax_max --compset I1PtClm51Bgc --res CLM_USRDAT --driver nuopc --user-mods-dir NEON/TALL

## Making modifiactions to the new cases and submitting the runs
This is going to take a while to finish. Depending with the power of your pc. This same procedure can be applied both using docker or using the super compute but make sure you modify the file paths accordingly

In [ ]:
%%bash
cd /home/user/TALLvcmax_min/
# Appending the new parameter file to the user_nl_clm
pfile="/home/user/inputdata/lnd/clm2/paramdata/ctsm51_params.c211112_vcmax_min.nc"
pfilestr="paramfile = '"$pfile"'"
echo -e "\n"$pfilestr >> user_nl_clm
# Now we need to make some modifications to our case before submitting
./xmlchange RUN_STARTDATE=2018-01-01
./xmlchange STOP_N=36
./xmlchange STOP_OPTION=nmonths

# we repeat the same procedure for the vcmax modified parameter file.
cd /home/user/TALLvcmax_max/
# Appending the new parameter file to the user_nl_clm
pfile="/home/user/inputdata/lnd/clm2/paramdata/ctsm51_params.c211112_vcmax_max.nc"
pfilestr="paramfile = '"$pfile"'"
echo -e "\n"$pfilestr >> user_nl_clm
# Now we need to make some modifications to our case before submitting
./xmlchange RUN_STARTDATE=2018-01-01
./xmlchange STOP_N=36
./xmlchange STOP_OPTION=nmonths

In [ ]:
%%bash
# Prepares the case, build and submit for vcmax_min case
cd /home/user/TALLvcmax_min/
./case.setup
#./case.build --clean
./case.build
./case.submit

In [ ]:
%%bash
# we repeat the same procedure for the vcmax modified parameter file.
cd /home/user/TALLvcmax_max/
./case.setup
#./case.build --clean
./case.build
./case.submit

## You have successfully finished creating and running the jobs.
You can use the visualization notebook to analyze your results